# Description of variational inference approach to estimating a DPGMM

We assume that the data follows a Dirichlet process Gaussian mixture model:

$$
\begin{aligned}
\theta_k | \alpha_\circ & \sim \text{Beta}(1,\alpha_\circ) \equiv p(\theta) & [\forall k=1,\ldots,\infty]\\
\pi_k | \theta_1,\ldots,\theta_k & = \theta_k\prod_{\ell=1}^{k-1}(1-\theta_\ell) & [\forall k=1,\ldots\infty]\\
\sigma^2_k | \beta_\circ, \gamma_\circ & \sim \text{InvGamma}(\beta_\circ, \gamma_\circ) \equiv p(\sigma^2) & [\forall k=1,\ldots,\infty] \\
\mu_k | \mu_\circ, \sigma^2_\circ & \sim \text{Normal}(\mu_\circ, \sigma^2_\circ) \equiv p(\mu) & [\forall k=1,\ldots,\infty] \\
k_i | \pi_1,\ldots,\pi_\infty & \sim \text{Categorical}(\pi_1,\ldots,\pi_\infty) & [\forall i\in\mathcal{I}]\\
x_i | k_i, \mu_{k_i}, \sigma^2_{k_i} & \sim \text{Normal}(\mu_{k_i}, \sigma^2_{k_i}) & [\forall i\in\mathcal{I}]
\end{aligned}
$$

The data generating process above makes use of the stick-breaking representation of the Dirichlet process. Given hyperpriors $(\alpha_\circ, \beta_\circ, \gamma_\circ, \mu_\circ, \sigma^2_\circ)$ and data $\{x\}_{i\in\mathcal{I}}\equiv\mathcal{D}$, our goal is to infer the posterior distribution over the model. This repository presents a variational inference (VI) approach towards this end.

A reference worth pointing out is [Blei and Jordan (2006)](https://projecteuclid.org/download/pdf_1/euclid.ba/1340371077), which introduces a truncated generalized stick-breaking process as a variational posterior for the Dirichlet process prior. We follow this approach but, importantly, we make use of the reparameterization trick to estimate the gradient of the evidence lower bound (ELBO) with respect to each parameter and carry out optimization via stochastic gradient ascent, whereas the cited paper uses a more involved coordinate ascent scheme.

The proposed variational distribution is given by:

$$
\begin{aligned}
\tilde{\theta}_k | a_k, b_k & \sim \text{Beta}(a_k, b_k) \equiv q(\tilde{\theta}_k|a_k,b_k) & [\forall k=1,\ldots,K-1]\\
\tilde{\pi}_k | \tilde{\theta}_1,\ldots,\tilde{\theta}_k & = \tilde{\theta}_k\prod_{\ell=1}^{k-1}(1-\tilde{\theta}_\ell) & [\forall k=1,\ldots,K-1]\\
\tilde{\pi}_K | \tilde{\pi}_1,\ldots\tilde{\pi}_{K-1} & = 1 - \sum_{k=1}^{K-1}\tilde{\pi}_k \\ 
\tilde{\sigma}^2_k | c_k, d_k & \sim \text{InvGamma}(c_k, d_k) \equiv q(\tilde{\sigma}^2_k|c_k,d_k)& [\forall k=1,\ldots,K] \\
\tilde{\mu}_k | e_k, f_k & \sim \text{Normal}(e_k, f_k) \equiv q(\tilde{\mu}_k|e_k,f_k) & [\forall k=1,\ldots,K] \\ 
\tilde{k}_i | \tilde{\pi}_1,\ldots,\tilde{\pi}_K & \sim \text{Categorical}(\tilde{\pi}_1,\ldots,\tilde{\pi}_K) & [\forall i\in\mathcal{I}] \\
\tilde{x}_i | \tilde{k}_i, \tilde{\mu}_{\tilde{k}_i}, \tilde{\sigma}^2_{\tilde{k}_i} & \sim \text{Normal}(\tilde{\mu}_{\tilde{k}_i}, \tilde{\sigma}^2_{\tilde{k}_i}) & [i\in\mathcal{I}]
\end{aligned}
$$

Hence, the ELBO is simply:

$$
\begin{aligned}
\text{ELBO} = \mathbb{E}_q & \left[\sum_{x\in\mathcal{D}}\log\left(\sum_{k=1}^K\exp(\log(\tilde{\pi}_k\cdot p(x|\tilde{\mu}_k,\tilde{\sigma}_k^2)))\right)\right] + \sum_{k=1}^{K-1}\text{KL}[q(\tilde{\theta}_k|a_k, b_k)\|p(\theta)] + \sum_{k=1}^{K}\text{KL}[q(\tilde{\sigma}^2_k|c_k, d_k)\|p(\sigma^2)] + \sum_{k=1}^{K}\text{KL}[q(\tilde{\mu}_k|e_k, f_k)\|p(\mu)].
\end{aligned}
$$

We approximate the expectation above with Monte Carlo integration and gradients follow directly via application of the reparameterization trick. As a practical matter, proper initialization of the variational parameters is important since the problem is nonconvex. In the self-contained example, we set $a_k=1$ and $b_k=100$ so that gradients can flow to each cluster at the early stages of training; all other variational parameters are initialized to values drawn from a mean zero normal distribution with small variance. Another thing worth noting is that many of the variational parameters are required to be constrained -- for instance, parameters of the inverse gamma distribution must be positive. These constraints are addressed via reparameterization, e.g., if a variable is restricted to the positive reals, we use an invertible mapping $h: \mathbb{R}\to\mathbb{R}_+$ (e.g., the softplus function) such that optimization is carried out over an unconstrained variable which, when passed through $h$, satisfies the required constraint.